In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv("heart.csv")

# Features and target variable
X = data.drop(columns=['HeartDisease'])  # Remove the target column from feature

In [10]:
from flask import Flask, request, jsonify, render_template
import numpy as np
import pandas as pd
import pickle
from threading import Thread

# Load the trained model and preprocessor
with open("heart_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("preprocessor.pkl", "rb") as f:
    preprocessor = pickle.load(f)

app = Flask(__name__)

# Route to render the HTML form
@app.route("/")
def home():
    return render_template("index.html")

# API endpoint to handle predictions
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        raw_features = data["features"]

        # Convert numerical values from string to proper types
        processed_features = [
            int(raw_features[0]),  # Age
            raw_features[1],        # Gender (string, remains unchanged)
            raw_features[2],        # Chest Pain Type (string, remains unchanged)
            int(raw_features[3]),  # Resting BP
            int(raw_features[4]),  # Cholesterol
            int(raw_features[5]),  # Fasting BS (0 or 1)
            raw_features[6],        # Resting ECG (string, remains unchanged)
            int(raw_features[7]),  # Max HR
            raw_features[8],        # Exercise Angina (string, remains unchanged)
            float(raw_features[9]), # Oldpeak (decimal value)
            raw_features[10]        # ST Slope (string, remains unchanged)
        ]
        features=processed_features

        # Convert to DataFrame for transformation
        features_df = pd.DataFrame([processed_features], columns=X.columns)

        # Transform using the preprocessor
        transformed_features = preprocessor.transform(features_df)

        # Make prediction
        prediction = model.predict(transformed_features)[0]
        probability = model.predict_proba(transformed_features)[0][1]

        return jsonify({
            "message": "appointment is booked for you" if int(prediction) else "u don't need a doctor",
            "prediction": int(prediction),
            "probability": float(probability)
        })
    except Exception as e:
        return jsonify({"error": str(e)})


# Function to run Flask server in a separate thread
def run():
    app.run(host="0.0.0.0", port=8082)

# Start Flask server
thread = Thread(target=run)
thread.start()


In [11]:
import requests

# Define the URL of your Flask API
url = "http://127.0.0.1:8082/predict"  # Localhost URL

# Example test data (modify based on your model's features)
test_data = {
    "features": [37,'M','ATA',130,283,0,'ST',98,'N',0,'Up']
}

# Send POST request to the API
response = requests.post(url, json=test_data)

# Print the response from the API
print(response.json())


{'message': "u don't need a doctor", 'prediction': 0, 'probability': 0.03517574684483672}
